In [1]:
import yaml
import base64
cred = yaml.load(base64.b64decode(open('/home/v.baydin/.secrets/dremio-tlinc.yaml.b64','r').read()), Loader=yaml.BaseLoader)
uid = cred['DREMIO_USER']
pwd = cred['DREMIO_PASSWORD']
import os
os.environ['DREMIO_USER'] = uid
os.environ['DREMIO_PASSWORD'] = pwd

import pandas as pd
from tl_datasource.dremio import get_async_dremio_connection,read_sql_to_pandas_aioodbc
from tl_pandas_utils.sql_to_parquet import read_sql_to_pyarrow_chunked,read_sql_to_parquet

# Скачиваем допуслуги в группировке по дням, направлениям, типу продукта, и pos

In [ ]:
async with get_async_dremio_connection() as conn:
    query = f'''select product_type,
       to_char(
               date_trunc('DAY', cast(creation_date as timestamp)),
               'yyyy-mm-dd'
           )         as product_creation_date_hour,
       departure,
       arrival,
       pos,
       --      sum(gap_profit_with_VAT) as sum_gap_profit_with_VAT,
       sum(price)    as sum_price,
       sum(quantity) as sum_quantity,
       count(*)      as count_rows
from "sp_IC.TR".stat_ancillaries_total
where creation_date > '2020-06-01'
  and not redemption_flag
group by product_type, date_trunc('DAY', cast(creation_date as timestamp)), departure, arrival, pos

    '''
    await read_sql_to_parquet('./ancillary_stats_from_june_2020.parquet', query, conn)


# Скачиваем допуслуги в группировке по часам, направлениям, типу продукта, и pos

In [5]:
async with get_async_dremio_connection() as conn:
    query = f'''
        select product_type,
               to_char(
                   date_trunc ('HOUR' , cast(creation_date as timestamp)),
                    'yyyy-mm-dd hh24:mi:ss'
                   )  as product_creation_date_hour,
               departure, arrival, pos,
         --      sum(gap_profit_with_VAT) as sum_gap_profit_with_VAT,
               sum(price) as sum_price,
               sum(quantity) as sum_quantity,
               count(*) as count_rows
        from "sp_IC.TR".stat_ancillaries_total
        where creation_date > '2020-06-01'
        and not redemption_flag
        group by product_type, date_trunc ('HOUR' , cast(creation_date as timestamp)), departure, arrival, pos

    '''
    await read_sql_to_parquet('./ancillary_stats_from_june_2020_by_hour.parquet', query, conn)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Скачиваем допуслуги из ibe3 в группировке по часам, направлениям, типу продукта, и pos

In [6]:
async with get_async_dremio_connection() as conn:
    query = f'''
        select product_type,
                              to_char(
                   date_trunc ('HOUR' , cast(creation_date as timestamp)),
                    'yyyy-mm-dd hh24:mi:ss'
                   )  as product_creation_date_hour,
               product_status, pos,
               sum(total_price) as sum_total_price,
               sum(seg_count) as sum_seg_count,
               count(*) as count_rows
        from "sp_IC.TR".stat_product_ibe3
        where creation_date > '2020-06-01'
        and not redemption_flag
        group by product_type, date_trunc ('HOUR' , cast(creation_date as timestamp)), product_status,pos

    '''
    await read_sql_to_parquet('./agg_stat_product_ibe3_from_june_2020_hour.parquet', query, conn)
    # infare = await read_sql_to_pandas_aiodbc(query,conn) 
#     q = read_sql_to_pyarrow_chunked(query, conn,return_percentage=True)
#     print(q)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Скачиваем проданные билеты (breakdown из ibe3) в группировке по дням, направлениям, статусу продукта, и pos

In [3]:
async with get_async_dremio_connection() as conn:
    query = f'''
        select to_char(
                   date_trunc ('DAY' , cast(creation_date as timestamp)),
                    'yyyy-mm-dd'
                   )  as product_creation_date,
               product_status, pos, itinerary,
               sum(total_price) as sum_total_price,
               sum(seg_count) as sum_seg_count,
               count(*) as count_rows
        from "sp_IC.TR".stat_product_ibe3
        where creation_date > '2020-06-01' and product_type='breakdown'
        and not redemption_flag
        group by date_trunc ('DAY' , cast(creation_date as timestamp)), product_status, pos, itinerary

    '''
    await read_sql_to_parquet('./agg_stat_breakdown_ibe3_from_june_2020_day.parquet', query, conn)
    # infare = await read_sql_to_pandas_aiodbc(query,conn) 
#     q = read_sql_to_pyarrow_chunked(query, conn,return_percentage=True)
#     print(q)

/home/v.baydin/Projects/innovations/venv/lib/python3.8/site-packages/tl_pandas_utils/sql_to_parquet.py:88: UserWarning: Streamer is iterated outside of its context
  async for i, table in aiostream.stream.enumerate(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Разбиваем itinerary на сегменты и агрегируем по ним

In [ ]:
df = pd.read_parquet('./agg_stat_breakdown_ibe3_from_june_2020_day.parquet')


Оставляем только completed билеты

In [3]:
df = df[df.product_status=='COMPLETED']
df

,product_creation_date,product_status,pos,itinerary,sum_total_price,sum_seg_count,count_rows
2,2020-08-19,COMPLETED,IBE,DME-OVB-KHV,103753.0,12,6
4,2021-01-01,COMPLETED,None,OVB-DME-ROV-DME-OVB,16140.0,4,1
5,2020-10-28,COMPLETED,IBE,DME-PEE,65612.0,22,22
6,2021-01-12,COMPLETED,BUS,AYT-DME-ROV,10756.0,2,1
7,2020-07-24,COMPLETED,IBE,OMS-DME,61320.0,10,10
...,...,...,...,...,...,...,...
4890772,2021-04-18,COMPLETED,MYB-mobile,GDZ-DME-OVB,12608.0,2,1
4890773,2021-03-16,COMPLETED,MYB-android,GOJ-GOJ-OVB,7500.0,2,1
4890775,2021-10-07,COMPLETED,sbr,UFA-DME-VOG,6606.0,2,1
4890776,2021-09-02,COMPLETED,MYB-iph,NJC-DME-SIP-DME-NJC,20020.0,8,2


In [10]:
import tqdm
from collections import defaultdict, namedtuple
from dataclasses import dataclass

Key = namedtuple('Key', ['product_creation_date', 'pos','origin', 'destination'])
@dataclass
class Stat:
    sum_total_price: float = 0
    sum_seg_count: int = 0
    count_rows:int = 0
    def __add__(self, other):
        return Stat(
            sum_total_price=self.sum_total_price+other.sum_total_price,
            sum_seg_count=self.sum_seg_count+other.sum_seg_count,
            count_rows=self.count_rows+other.count_rows
        )

In [19]:
def extract_itinerary(row):
    it = row.itinerary
    if it is None:
        return []
    airports = it.split('-')
    origins = [airports[i] for i in range(len(airports)-1)]
    destinations = [airports[i+1] for i in range(len(airports)-1)]
    rows=[]
    for origin, dest in zip(origins,destinations):
        key = Key(
            product_creation_date=row.product_creation_date,
            pos=row.pos,
            origin=origin,
            destination=dest
        )
        rows.append(
            (key,
             Stat(
                 sum_total_price=row.sum_total_price,
                 sum_seg_count=row.sum_seg_count,
                 count_rows=row.count_rows
            ))
        )
    return rows

extracted = defaultdict(Stat)
for _,row in tqdm.tqdm(df.iterrows(),total=len(df),):
    rows = extract_itinerary(row)
    for k,v in rows:
        extracted[k]+=v


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3263341/3263341 [06:04<00:00, 8941.54it/s]


In [20]:
len(extracted)

1619492

In [22]:
processed_df = pd.DataFrame.from_dict(extracted, orient='index')

idx = pd.DataFrame(processed_df.index.tolist())

final_df = pd.concat([idx,processed_df.reset_index()], axis=1).drop(columns=['index']).sort_values('product_creation_date')

final_df.to_parquet('./agg_stat_breakdown_ibe3_from_june_2020_day_extracted.parquet')